# Trying out an LDA model to apply topic modelling to MEAs

In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"


In [2]:
# Handle imports
import numpy as np
import pandas as pd
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
import pickle
from gensim.models import CoherenceModel
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import string
import pprint
import pyLDAvis.gensim_models as gensimvis
from tqdm import tqdm_notebook as tqdm

/opt/anaconda3/envs/APL2021_DataPlus/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

/opt/anaconda3/envs/APL2021_DataPlus/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
nlp = spacy.load('en_core_web_lg')
lemmatizer = nlp.get_pipe("lemmatizer")

/opt/anaconda3/envs/APL2021_DataPlus/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
stop_word_list = [
    "transcript",
    "respondent",
    "ohio",
    "division",
    "department",
    "commerce",
    "financial",
    "revised",
    "code",
    "institutions",
    "chapter",
    "codified",
    "institution",
    "heshe",
    "rc",
    "oh",
    "mortgage",
    "fact",
    "notice",
    "act",
    "chapters",
    "arizona",
    "january",
    "february",
    "april",
    "march",
    "may",
    "june",
    "july",
    "august",
    "september",
    "october",
    "november",
    "december",
    "dfi",
    "ce",
    "pursuant",
    "realize",
    "days",
    "result",
    "investigation",
    "indicate",
    "state",
    "address",
    "appropriate",
    "broker",
    "business",
    "certificate",
    "charge",
    "codify",
    "consistent",
    "date",
    "enforce",
    "find",
    "hearing",
    "mail",
    "mailing",
    "matter",
    "necessary",
    "notify",
    "order",
    "purpose",
    "record",
    "registration",
    "request",
    "responsibility",
    "return",
    "right",
    "serve",
    "loan",
    "officer",
    "application",
    "background",
    "come",
    "attorney",
    "law",
    "duly",
    "appoint",
    "accordance",
    "revise",
    "administrative",
    "detailed",
    "paragraph",
    "page",
    "exhibit",
    "robert",
    "grieser",
    "superintendent",
    "bob",
    "taft",
    "jennifer",
    "protect",
    "proscribe",
    "\\t",
    "m",
    'allege',
    'allegation',
    'appellate',
    'case',
    'authorize',
    'applicant',
    'denial',
    'settlement',
    'agreement',
    'report',
    'recommendation',
    'following',
    'stipulation',
    'admission',
    'finding',
    'findings',
    'basis',
    'administer',
    'amend',
    'amendment',
    'bill',
    'understanding',
    'empower',
    'formal',
    'proceeding',
    'institute',
    'terminate',
    'intent',
    'originate',
    'section',
    'rule',
    'adjucative',
    'hereinafter',
    'motion',
    'jurisdiction',
    'continue',
    'file',
    'opportunity',
    'registrant',
    'conclusion',
    'andor',
    'consideration',
    'wherefore',
    'lieu',
    'specifically',
    'expressly',
    'condition',
    'hearma',
    'damel',
    'broxterman',
    'practice',
    'procedure',
    'time',
    'thereof',
    'attest',
    'tr',
    'p',
    'consider',
    'consist',
    'reserve',
    'occur',
    'e',
    'b',
    'licensee',
    '31st',
    'previously',
    'apply',
    'understand',
    'originator',
    'transfer',
    'dollar',
    'operation',
    'manager',
    'sections',
    'municipal',
    'constitute',
    'evidence',
    'adjudicatory',
    'defendant',
    'agree',
    'provide',
    'provision',
    'follow',
    'ace',
    'adopt',
    'applicable'
]

/opt/anaconda3/envs/APL2021_DataPlus/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
len(stop_word_list)

/opt/anaconda3/envs/APL2021_DataPlus/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


181

In [7]:
nlp.Defaults.stop_words.update(stop_word_list)
# Iterates over the words in the stop words list and resets the "is_stop" flag.
for word in STOP_WORDS:
    lexeme = nlp.vocab[word]
    lexeme.is_stop = True
lemma_tags = {"NNS", "NNPS"}

/opt/anaconda3/envs/APL2021_DataPlus/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
# Store word_list in txt file for easier reference

with open('stop_words.txt','w') as file:
    file.writelines("%s\n" % word for word in nlp.Defaults.stop_words)

len(nlp.Defaults.stop_words)

/opt/anaconda3/envs/APL2021_DataPlus/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


506

In [9]:
# Load data
df = pd.read_csv("Standardized Data/Ohio_Final_cleaned.csv")

/opt/anaconda3/envs/APL2021_DataPlus/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
def clean_text(sample_text):
    sample_text = sample_text.translate(str.maketrans('', '', string.punctuation))
    sample_text = " ".join([w for w in sample_text.split() if not w.isdigit()])
    sample_text = sample_text.lower()
    doc = nlp(sample_text)
    doc = [token.lemma_ for token in doc if (token.tag_ in lemma_tags or token.lemma_ != '-PRON-' and token.pos_ in ['NOUN', 'ADJ', 'VERB', 'ADV'])]
    doc = u' '.join(doc)
    doc = nlp.make_doc(doc)
    tokens = [token.text for token in doc if token.is_stop != True and token.is_punct != True]
    return tokens

/opt/anaconda3/envs/APL2021_DataPlus/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
sample_text = df.loc[4590,"Text"]
print(clean_text(sample_text))
print(len(clean_text(sample_text)))

/opt/anaconda3/envs/APL2021_DataPlus/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['accountable', 'licensing', 'individual', 'residential', 'hold', 'license', 'number', 'issue', 'birth', 'employer', 'require', 'suspend', 'prior', 'hear', 'license', 'fail', 'fulfill', 'education', 'requirement', 'require', 'education', 'complete', 'fine', 'pay', 'treasurer', 'credit', 'consumer', 'finance', 'fund', 'c4', 'individual', 'suspend', 'license', 'permit', 'residential', 'hold', 'license', 'calendar', 'year', 'fail', 'fulfill', 'education', 'requirement', 'calendar', 'year', 'license', 'number', 'issue', 'suspend', 'license', 'suspension', 'hear']
52


In [12]:
sample_text

/opt/anaconda3/envs/APL2021_DataPlus/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


" WHEREAS, the Ohio Department of Commerce, by and through the Superintendent of the Division of Financial Institutions (Division), is charged with the responsibility of administering and enforcing the Ohio Mortgage Broker Act, codified in Ohio Revised Code (R.C.) Chapter 1322. In accordance therewith, the Division is accountable for the licensing of individuals as residential mortgage loan officers; and WHEREAS, DeLise S. Simmons (Respondent) holds a loan officer license number 002825 issued by the Division. Her address of record is 914 Lincoln Avenue, Apt. 2, Toledo, Ohio 43607, and her date of birth is April 1, 1970. Respondent's employer of record is MJM Development, Inc. d/b/a American Eagle Funding, 3962-A Brown Park Road, Hilliard, Ohio 43026; and WHEREAS, R.C. 1322.10 (F)(2) requires the Division to suspend, without a prior hearing, the license of a licensee who failed to fulfill the continuing education requirements of R.C. 1322.052 until such time as the required continuing e

In [21]:
text = df["Text"]
text_list = []
for doc in tqdm(text):
    if not pd.isna(doc):
        pr = clean_text(doc)
        text_list.append(pr)

/opt/anaconda3/envs/APL2021_DataPlus/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-21-470133570a00>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for doc in tqdm(text):


  0%|          | 0/6918 [00:00<?, ?it/s]

In [ ]:
# Store word_list in txt file for easier reference

with open('ohio_text_list.txt','w') as file:
    file.writelines("%s\n" % text for text in text_list)

In [16]:
text_list = []
with open('ohio_text_list.txt','r', encoding = "ISO-8859-1") as file:
    file_contents = file.readlines()
    for line in file_contents:
        current_text = line[:-1]
        text_list.append(current_text)

/opt/anaconda3/envs/APL2021_DataPlus/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
len(sample_list)

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(text_list, min_count=5, threshold=30) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[text_list], threshold=30) 
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)

trigram_mod = gensim.models.phrases.Phraser(trigram)
# See trigram example

print(trigram_mod[bigram_mod[text_list[1]]])

In [ ]:
grammed_list = [trigram_mod[bigram_mod[doc]]for doc in text_list]

In [22]:
# RUN NEXT 
#Creates, which is a mapping of word IDs to words.
words = corpora.Dictionary(text_list)

# Turns each document into a bag of words.
corpus = [words.doc2bow(doc) for doc in text_list]

/opt/anaconda3/envs/APL2021_DataPlus/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Creates, which is a mapping of word IDs to words.
grammed_words = corpora.Dictionary(grammed_list)

# Turns each document into a bag of words.
grammed_corpus = [words.doc2bow(doc) for doc in grammed_list]

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=words, random_state = 12)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
model = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=19, id2word=words, random_state = 12)

In [ ]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=words, corpus=corpus, texts=text_list, start=10, limit=15, step=1)

In [ ]:
# Can take a long time to run.
grammed_model_list, grammed_coherence_values = compute_coherence_values(dictionary=grammed_words, corpus=grammed_corpus, texts=grammed_list, start=18, limit=25, step=1)

In [17]:
import tomotopy as tp
import numpy as np

term_weight = tp.TermWeight.ONE
hdp = tp.HDPModel(tw = term_weight, min_cf =5, rm_top = 7, gamma = 1, alpha = 0.1,initial_k = 10, seed = 12)

/opt/anaconda3/envs/APL2021_DataPlus/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
sum = 0
for text in text_list:
    if len(text) > 0:
        hdp.add_doc(text)
print(sum)
# Initiate sampling burn-in  (i.e. discard N first iterations)
hdp.burn_in = 100
hdp.train(0)
print('Num docs:', len(hdp.docs), ', Vocab size:', hdp.num_vocabs,
      ', Num words:', hdp.num_words)
print('Removed top words:', hdp.removed_top_words)

# Train model
for i in range(0, 1000, 100):
    hdp.train(100) # 100 iterations at a time
    print('Iteration: {}\tLog-likelihood: {}\tNum. of topics: {}'.format(i, hdp.ll_per_word, hdp.live_k))

In [ ]:
model = model_list[5].save("lda2.model")
model2= gensim.models.ldamodel.LdaModel.load("lda2.model")

Start of Code that Needs to be Run

In [18]:
# RUN THIS LINE TO SELECT THE MODEL

model_selected = gensim.models.ldamodel.LdaModel.load("lda1.model")



/opt/anaconda3/envs/APL2021_DataPlus/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
term_weight = tp.TermWeight.ONE
hdp_gram = tp.HDPModel(tw = term_weight, min_cf =5, rm_top = 7, gamma = 1, alpha = 0.1,initial_k = 10, seed = 12)

In [ ]:
sum = 0
for text in grammed_list:
    if len(text) > 0:
        hdp_gram.add_doc(text)
print(sum)
# Initiate sampling burn-in  (i.e. discard N first iterations)
hdp_gram.burn_in = 100
hdp_gram.train(0)
print('Num docs:', len(hdp_gram.docs), ', Vocab size:', hdp_gram.num_vocabs,
      ', Num words:', hdp_gram.num_words)
print('Removed top words:', hdp_gram.removed_top_words)

# Train model
for i in range(0, 1000, 100):
    hdp_gram.train(100) # 100 iterations at a time
    print('Iteration: {}\tLog-likelihood: {}\tNum. of topics: {}'.format(i, hdp_gram.ll_per_word, hdp_gram.live_k))

In [ ]:
# Show graph
limit=28; start=20; step=1;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [23]:
# EVERYTHING AFTER THIS CAN BE RUN TO VISUALIZE STUFF AND SEE CATEGORIES

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

# feed the LDA model into the pyLDAvis instance
gensimvis.prepare(model_selected, corpus, words)

/opt/anaconda3/envs/APL2021_DataPlus/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.254015 -0.038910       1        1  19.321489
15    -0.078878  0.207795       2        1  14.421356
2      0.158720  0.096938       3        1   9.106210
16    -0.140545  0.004482       4        1   8.997182
11     0.256264  0.020816       5        1   8.307409
17    -0.028587 -0.163046       6        1   6.318150
14    -0.067754  0.106117       7        1   5.927834
7     -0.100386 -0.000459       8        1   5.144287
10    -0.106434  0.144804       9        1   4.341552
0     -0.054216 -0.120887      10        1   2.806158
4      0.005643  0.112518      11        1   2.788276
18    -0.162664  0.123176      12        1   2.627973
6      0.173418 -0.050752      13        1   2.505143
3      0.077854 -0.028178      14        1   2.010011
9      0.094003 -0.028113      15        1   1.199895
19    -0.080607 -0.025990      16        1   1.177998
5     -0.159786 -0.204553      17        1   1.174872
13    -0.106613 -0.100895      18        1   1.064887
1      0.091509  0.031408      19        1   0.707838
12    -0.024957 -0.086269      20        1   0.051479, topic_info=            Term          Freq         Total Category  logprob  loglift
231      license  27431.000000  27431.000000  Default  30.0000  30.0000
429      renewal   5892.000000   5892.000000  Default  29.0000  29.0000
25          fail  10787.000000  10787.000000  Default  28.0000  28.0000
1196    calendar   6132.000000   6132.000000  Default  27.0000  27.0000
1124  suspension   2177.000000   2177.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
358       submit      0.964009   4827.950518  Topic20  -5.8047  -0.9471
1196    calendar      0.963451   6132.418940  Topic20  -5.8053  -1.1868
114         base      0.750105   1548.969232  Topic20  -6.0556  -0.0611
196      general      0.855816   5058.997969  Topic20  -5.9238  -1.1129
5        conduct      0.851417   6974.583879  Topic20  -5.9289  -1.4391

[1351 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
4806      4  0.239051  040279lod
4806     12  0.478101  040279lod
478      19  0.376941  13018703c
433      10  0.831823    132153b
1381      1  0.079275   1322041b
...     ...       ...        ...
395      18  0.004059       year
395      19  0.014696       year
395      20  0.000280       year
1884      2  0.246287    yohance
1884     15  0.615718    yohance

[7326 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 16, 3, 17, 12, 18, 15, 8, 11, 1, 5, 19, 7, 4, 10, 20, 6, 14, 2, 13])

In [24]:
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

/opt/anaconda3/envs/APL2021_DataPlus/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
def format_topics_sentences(ldamodel=model_selected, corpus=corpus, texts=text_list):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=model_selected, corpus=corpus, texts=text_list)
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()
topic_counts.head()
# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics

df_dominant_topics.sort_values(by = ["Num_Documents"],ascending = False, inplace = True)
df_dominant_topics["Old Index"] = df_dominant_topics.index 
df_dominant_topics.reset_index(inplace = True)
mapping = df_dominant_topics[["Old Index"]].to_dict()['Old Index']
mapping = {value : key+1 for (key, value) in mapping.items()}
mapping

/opt/anaconda3/envs/APL2021_DataPlus/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{15.0: 1,
 11.0: 2,
 2.0: 3,
 8.0: 4,
 4.0: 5,
 6.0: 6,
 16.0: 7,
 17.0: 8,
 7.0: 9,
 10.0: 10,
 14.0: 11,
 9.0: 12,
 0.0: 13,
 13.0: 14,
 3.0: 15,
 18.0: 16,
 19.0: 17,
 5.0: 18,
 1.0: 19}

In [26]:
df_topic_sents_keywords["Dominant_Topic"] = df_topic_sents_keywords["Dominant_Topic"].apply(lambda x: mapping[x])

/opt/anaconda3/envs/APL2021_DataPlus/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show topics per text
df_dominant_topic

/opt/anaconda3/envs/APL2021_DataPlus/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,18,0.8879,"consumer, disclosure, document, violation, taxis, relate, statement, cause, letter, borrower","[conduct, affair, incorporator, statutory, agent, president, joint, action, cause, customer, proceed, excess, disburse, mean, fictitious, creditor, customer, relate, document, joint, action, cause, customer, proceed, excess, intend, customer, legitimate, creditor, customer, convert, knowledge, improper, fraudulent, dishonest, fail, implement, review, control, measure, design, prevent, future, improper, fraudulent, dishonest, joint, action, fail, disclose, fee, violation, federal, real, estate, joint, action, receive, customer, proceed, disbursement, fictitious, creditor, disbursement, receipt, unearned, fee, thing, value, violation, federal, real, estate, joint, action, devise, intend, devise, scheme, artifice, defraud, obtain, money, mean, false, fraudulent, pretense, representation, transmit, cause, transmit, writing, form, federal, real, estate, document, relate, document, mean, ...]"
1,1,9,0.5799,"bond, testimony, license, surety, company, hold, letter, submit, issue, question","[abovecaptione, license, commence, office, south, high, floor, hold, close, day, hold, funding, crew, revoke, inter, alia, base, engage, improper, fraudulent, dishonest, activity, appear, represent, general, general, appear, represent, issue, deny, refer, 01mb11, service, perfect, timely, schedule, requirement, orc, original, counsel, new, counsel, forward, posthearing, ruling, offer, remain, 3h, letter, apparently, sign, crew, letterhead, letter, white, testimony, testimony, letter, examination, additional, testimony, include, testimony, opposition, offer, leave, determine, admit, use, weight, document, 3h, appear, belong, witness, appear, similarity, likely, obtain, document, simply, filing, error, introduce, reason, include, demonstrate, activity, involve, client, letter, accessible, review, argument, propose, limit, review, issue, introduce, identify, ...]"
2,2,13,0.3784,"cease, license, residential, issue, conduct, uniform, require, summary, document, cancellation","[public, public, interest, send, certify, revoke, hear, crew, funding, hold, daysjuly, issue, determine, crew, funding, violate, list, fictitious, creditor, false, information, document, convert, borrower, creditor, money, cause, customer, sign, blank, document, excess, fee, interfere, crew, funding, recommend, revoke, objection, carefully, objection, thereto, reject, objection, confirm, approve, issue, crew, funding, revoke]"
3,3,9,0.3644,"bond, testimony, license, surety, company, hold, letter, submit, issue, question","[send, services, corporation, certify, deny, receive, inform, intention, issue, deny, base, determination, fail, information, conclude, meet, education, experience, requirement, response, expire, recently, contact, secretary, vice, president, inform, long, operate, receive, fax, issue, withdraw, intention, issue, deny, moot, accordingly, services, corporation, termination, hear]"
4,4,3,0.5012,"license, issue, renewal, fail, deny, day, public, inform, conduct, refuse","[public, lending, interest, public, specialists, corporation, renew, period, certify, revoke, entitle, adjudicative, day, day, issue, revoke, specialists, specialists, fail, fail, defend, violate, substantial, misrepresentation, renewal, obtain, false, fraudulent, representation, contain, true, revoke]"
...,...,...,...,...,...
6302,6302,4,0.5931,"violation, omba, license, requirement, hour, complete, comply, year, calendar, fail","[enter, refuse, renew, second, lender, default, payment, annual, fee, prescribe, community, require, submit, renewal, annual, fee, community, submit, renewal, remain, pende, community, timely, submit, renewal, authority, refuse, renew, community, omla, enter, deny, community, renewal, untimely, filing, base, violation, noncompliance, omla, effective, forego, mutual, promise, set, forth, disciplinary, commu

In [28]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet

/opt/anaconda3/envs/APL2021_DataPlus/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,1,0.9971,"license, list, criminal, convict, offense, violate, statement, involve, prohibit, theft","[conduct, convict, unauthorized, use, property, misdemeanor, fourth, degree, crb, court, common, pleas, plead, guilty, convict, attempt, receive, steal, property, misdemeanor, degree, cr, c, convict, theft, misdemeanor, degree, crb, plead, guilty, convict, operate, motor, vehicle, intoxicate, trc, plead, guilty, convict, operate, motor, vehicle, intoxicate, misdemeanor, degree, deny, license, hear, plead, guilty, convict, operate, motor, vehicle, intoxicate, misdemeanor, degree, trc, plead, guilty, convict, assault, misdemeanor, degree, crb, h, swear, statement, information, criminal, license, submit, complete, truthful, attempt, obtain, license, untruthful, information, criminal, list, determine, action, list, prove, honest, truthful, good, reputation, believe, commit, criminal, offense, involve, theft, criminal, offense, involve, money, security, ...]"
1,2,0.9849,"license, fail, refuse, require, calendar, year, comply, issue, complete, hour","[conduct, refuse, issue, license, violate, fail, comply, conduct, hold, license, calendar, year, require, licensed, complete, hour, approve, education, calendar, year, d, year, impose, requirement, extend, deadline, comply, requirement, extension, grant, comply, requirement, fail, complete, require, hour, credit, extended, deny, license, renewal, hear, fail, comply, character, general, fitness, command, confidence, public, warrant, belief, operate, honestly, fairly, compliance, g, fail, comply, refuse, issue, license]"
2,3,0.9888,"license, issue, renewal, fail, deny, day, public, inform, conduct, refuse","[license, renewal, interest, public, kohler, submit, renewal, license, issue, inform, conduct, action, character, general, fitness, command, confidence, public, warrant, belief, operate, honestly, fairly, compliance, action, violate, prohibit, substantial, misrepresentation, license, action, violate, prohibit, false, misleading, statement, material, omission, statement, require, action, violate, prohibit, engage, conduct, improper, fraudulent, dishonest, dealing, fail, cooperate, violate, violate, license, deny, kohler, informed, deny, license, renewal, regard, day, far, caution, receive, write, office, day, issue, deny, renewal, license, certify, service, perfect, fail, fail, defend, contain, true, respondentd, renewal, license, deny, kohlers, license, renewal, deny]"
3,4,0.9954,"violation, omba, license, requirement, hour, complete, comply, year, calendar, fail","[enter, refuse, issue, license, thing, fail, comply, omba, thereunder, issue, license, thing, comply, omba, c, require, complete, hour, approve, education, calendar, year, hold, license, calendar, year, admit, fail, complete, require, credit, hour, calendar, year, fail, comply, authority, refuse, issue, license, enter, deny, license, admit, violation, noncompliance, omba, h, base, violation, noncompliance, omba, effective, forego, mutual, promise, set, forth, disciplinary, knowingly, voluntarily, term, limitation, penalty, violation, describe, pay, fine, payment, form, cashier, check, money, payable, consumer, finance, submit, complete, hour, approve, calendar, year, requirement, calendar, year, requirement, submit, copy, attendance, credit, hour, issue, provider, write, red, ink, face, copy, attendance, requirement, ...]"
4,5,0.9768,"license, objection, deny, hold, recommend, submit, suspension, testimony, copy, attach","[license, submit, license, issue, deny, thereto, hold, write, recommend, deny, attach, copy, letter, explain, submit, written, objection, objection, receive, objection, attach, objection, attach, objection, submit, appear, attempt, submit, additional, close, hear, attach, objection, objection, thereto, confirm, approve, accordingly, license, deny]"
5,6,0.9712,"renewal, license, issue, requirement, education, deny, refuse, termination, complete, require","[drive, t